<a href="https://colab.research.google.com/github/vasan12sp/LowLightImgEnhancementUsingPSO/blob/main/PSO_IMG_ENHANCEMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyswarm
!pip install brisque

import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pyswarm import pso
from skimage.metrics import structural_similarity as ssim
from skimage import img_as_ubyte
from scipy.stats import entropy
import os
import skimage
from skimage import img_as_float
from skimage.restoration import estimate_sigma
import brisque


  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4463 sha256=0c5d63f66b8a5cb77df46d8ff96d540ded7ae4280782d6778d53503ce6bf5044
  Stored in directory: /root/.cache/pip/wheels/bb/4f/ec/8970b83323e16aa95034da175454843947376614d6d5e9627f
Successfully built pyswarm
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 6.5 MB/s eta 0:00:00
  Created wheel for libsvm-official: filename=libsvm_official-3.35.0-cp311-cp311-linux_x86_64.whl size=124215 sha256=ef1419f731bd4f9cdba76ab645c0500b370cbc56073876c4b279b12190288162
  Stored in directory: /root/.cache/pip/wheels/cb/1b/c2/e2bf699257a3150a4c54c95e6efe70586ea1a4d97e71619076
Successfully built libsvm-official


In [ ]:

def load_lol_dataset(dataset_path):
    low_light_path = os.path.join(dataset_path, "our485/low")
    low_light_images = []

    for filename in os.listdir(low_light_path):
        img_path = os.path.join(low_light_path, filename)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Warning: Could not read {img_path}")
            continue  # Skip unreadable images

        img = cv2.resize(img, (400, 600))
        low_light_images.append(img)

    return np.array(low_light_images)




In [ ]:
def calculate_entropy(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist, _ = np.histogram(grayscale, bins=256, range=(0, 255), density=True)
    return entropy(hist)

def calculate_sharpness(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    return laplacian.var()

def calculate_brightness(image):
    return np.mean(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

def calculate_ssim(original, enhanced):
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    enhanced_gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
    return ssim(original_gray, enhanced_gray, data_range=255)

def calculate_psnr(original, enhanced):
    return cv2.PSNR(original, enhanced)

def calculate_ndf(image):
    obj = brisque.BRISQUE()
    return obj.score(image)  # Lower BRISQUE score = better quality


In [ ]:
def histogram_equalization(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

def enhance_image(image, params):
    contrast, brightness, gamma = params
    new_image = cv2.convertScaleAbs(image, alpha=contrast, beta=brightness)
    gamma_corrected = np.array(255 * (new_image / 255) ** gamma, dtype='uint8')
    return gamma_corrected

def objective_function(params, image):
    enhanced = enhance_image(image, params)

    e = calculate_entropy(enhanced)      # Maximize
    s = calculate_sharpness(enhanced)    # Maximize
    b = calculate_brightness(enhanced)   # Could be used optionally
    similarity = calculate_ssim(image, enhanced)  # Maximize
    quality = calculate_psnr(image, enhanced)  # Maximize
    ndf = calculate_ndf(enhanced)  # Minimize

    # Weights for each metric
    w1, w2, w3, w4, w5, w6 = 3, 3, 2, 3, 4, 5

    # Negative sign because PSO minimizes the function, so we negate it
    return -(w1 * e + w2 * s + w3 * b + w4 * similarity + w5 * quality - w6 * ndf)




In [ ]:
def optimize_image(image):
    lb = [0.5, -50, 0.5]  # Lower bounds for [contrast, brightness, gamma]
    ub = [2.0, 50, 2.5]   # Upper bounds
    best_params, _ = pso(objective_function, lb, ub, args=(image,), swarmsize=20, maxiter=10)
    return best_params




In [ ]:
def process_lol_dataset(dataset_path):
    low_light_images = load_lol_dataset(dataset_path)

    # Select 100 random images
    num_images = min(100, len(low_light_images))
    selected_images = random.sample(list(low_light_images), num_images)


    # Initialize accumulators for average calculation
    total_he_entropy = total_pso_entropy = 0
    total_he_sharpness = total_pso_sharpness = 0
    total_he_brightness = total_pso_brightness = 0
    total_he_ssim = total_pso_ssim = 0
    total_he_psnr = total_pso_psnr = 0
    total_he_ndf = total_pso_ndf = 0

    for i, low_light_img in enumerate(selected_images):
        # Histogram Equalization
        he_img = histogram_equalization(low_light_img)

        # PSO Enhancement
        best_params = optimize_image(low_light_img)
        pso_img = enhance_image(low_light_img, best_params)

        # Compute Metrics
        he_entropy = calculate_entropy(he_img)
        pso_entropy = calculate_entropy(pso_img)

        he_sharpness = calculate_sharpness(he_img)
        pso_sharpness = calculate_sharpness(pso_img)

        he_brightness = calculate_brightness(he_img)
        pso_brightness = calculate_brightness(pso_img)

        he_ssim = calculate_ssim(low_light_img, he_img)
        pso_ssim = calculate_ssim(low_light_img, pso_img)

        he_psnr = calculate_psnr(low_light_img, he_img)
        pso_psnr = calculate_psnr(low_light_img, pso_img)

        he_ndf = calculate_ndf(he_img)
        pso_ndf = calculate_ndf(pso_img)

        # Accumulate values for averaging
        total_he_entropy += he_entropy
        total_pso_entropy += pso_entropy
        total_he_brightness += he_brightness
        total_pso_brightness += pso_brightness
        total_he_ssim += he_ssim
        total_pso_ssim += pso_ssim
        total_he_psnr += he_psnr
        total_pso_psnr += pso_psnr
        total_he_ndf += he_ndf
        total_pso_ndf += pso_ndf

        # Print Metrics for each image
        print(f"\nImage {i+1} Metrics:")
        print(f"{'Metric':<15}{'Histogram Eq.':<15}{'PSO Enhanced'}")
        print(f"{'-'*50}")
        print(f"Entropy        {he_entropy:.4f}       {pso_entropy:.4f}")
        print(f"Brightness     {he_brightness:.4f}       {pso_brightness:.4f}")
        print(f"SSIM           {he_ssim:.4f}       {pso_ssim:.4f}")
        print(f"PSNR           {he_psnr:.4f}       {pso_psnr:.4f}")
        print(f"NDF            {he_ndf:.4f}       {pso_ndf:.4f}")

        # Display results
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        axes[0].imshow(cv2.cvtColor(low_light_img, cv2.COLOR_BGR2RGB))
        axes[0].set_title("Original Image")
        axes[1].imshow(cv2.cvtColor(he_img, cv2.COLOR_BGR2RGB))
        axes[1].set_title("Histogram Equalization")
        axes[2].imshow(cv2.cvtColor(pso_img, cv2.COLOR_BGR2RGB))
        axes[2].set_title("PSO Enhancement")
        plt.show()

    # Compute and display final average metrics
    avg_he_entropy = total_he_entropy / num_images
    avg_pso_entropy = total_pso_entropy / num_images
    avg_he_brightness = total_he_brightness / num_images
    avg_pso_brightness = total_pso_brightness / num_images
    avg_he_ssim = total_he_ssim / num_images
    avg_pso_ssim = total_pso_ssim / num_images
    avg_he_psnr = total_he_psnr / num_images
    avg_pso_psnr = total_pso_psnr / num_images
    avg_he_ndf = total_he_ndf / num_images
    avg_pso_ndf = total_pso_ndf / num_images

    print("\nFinal Average Metrics for 100 Randomly Selected Images:")
    print(f"{'Metric':<15}{'Histogram Eq.':<15}{'PSO Enhanced'}")
    print(f"{'-'*50}")
    print(f"Entropy        {avg_he_entropy:.4f}       {avg_pso_entropy:.4f}")
    print(f"Brightness     {avg_he_brightness:.4f}       {avg_pso_brightness:.4f}")
    print(f"SSIM           {avg_he_ssim:.4f}       {avg_pso_ssim:.4f}")
    print(f"PSNR           {avg_he_psnr:.4f}       {avg_pso_psnr:.4f}")
    print(f"NDF            {avg_he_ndf:.4f}       {avg_pso_ndf:.4f}")






In [ ]:
# Run the LOL dataset processing
process_lol_dataset("/kaggle/input/lol-dataset/lol_dataset/")